# `DATA EXPLORATION`

## **TOPIC: FILMS ANALYSIS**

`Group ID`: 17

`Group Member`:
- 22127404_Tạ Minh Thư
- 22127359_Chu Thúy Quỳnh
- 22127302_Nguyễn Đăng Nhân

### Read data

In [ ]:
file = open('films_data.csv', 'r', encoding='utf-8-sig')

data = {}
first_line = file.readline().strip().split('\t')
for val in first_line:
    data[val] = []

for line in file:
    line_vals = line.strip().split('\t')   
    for i in range(len(line_vals)):
        data[first_line[i]].append(line_vals[i])

{'Rank': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121',
  '122',
  

### The meaning of each row
Each row represents a specific movie, detailing information about its release, performance, genre, and key contributors (director, writer, and cast).

In [98]:
print('Number of rows:', len(data['Rank']))

Number of rows: 1000


### The meaning of each column
- `Ranks`: The film's rank in the top lifetime grosses.
- `Titles`: The film's name.
- `Foreign %`: The percentage of the foreign grosses in the film's worldwide grosses.
- `Domestic %`: The percentage of the domestic grosses in the film's worldwide grosses.
- `Years`: The year that the film was first released.
- `Genres`: The genre(s) associated with each film.
- `Directors`: The director(s) of each film.
- `Writers`: The writer(s) credited for each film.
- `Casts`: The main cast members of each film.

In [99]:
print('Number of columns:', len(data))

Number of columns: 9


### The datatype of each column

In [100]:
for col_name, col_value in data.items():
    print(f'{col_name}: {type(col_value[0])}')


Rank: <class 'str'>
Title: <class 'str'>
Foreign %: <class 'str'>
Domestic %: <class 'str'>
Year: <class 'str'>
Genre: <class 'str'>
Director: <class 'str'>
Writer: <class 'str'>
Cast: <class 'str'>


### Preprocessing data

- Convert Percentage Columns: Convert Foreign % and Domestic % to numeric values by removing the '%' symbol and changing the data type to floats. If the value in these columns is '-', it indicates that the foreign gross accounts for 100% of the film's worldwide grosses, and the domestic gross is considered 0%.

- Standardize Year Data Type: Ensure Year is an integer for easy numerical analysis.

- Split Genres: Split the values in Genre into separate columns or lists for better analysis of each genre individually.

- Director and Writer Parsing: If needed, split multiple directors or writers into lists to analyze individual contributions.

- Cast Parsing: Similarly, parse the Cast column into individual actor names or convert to lists, which will make it easier to analyze actor appearances across movies.

In [101]:
data['Rank'] = [int(rank.replace(',', '')) for rank in data['Rank']]

data['Foreign %'] = [float(value.rstrip('%')) for value in data['Foreign %']]

data['Domestic %'] = [
    float(value.replace('<', '').rstrip('%')) if value != '-' else 0 
    for value in data['Domestic %']
]

data['Year'] = [int(year) for year in data['Year']]

data['Genre'] = [genres.split(', ') for genres in data['Genre']]

data['Director'] = [directors.split(', ') for directors in data['Director']]

data['Writer'] = [writers.split(', ') for writers in data['Writer']]


data['Cast'] = [actors.split(', ') for actors in data['Cast']]


### New datatype of each column

In [102]:
for col_name, col_value in data.items():
    print(f'{col_name}: {type(col_value[0])}')

Rank: <class 'int'>
Title: <class 'str'>
Foreign %: <class 'float'>
Domestic %: <class 'float'>
Year: <class 'int'>
Genre: <class 'list'>
Director: <class 'list'>
Writer: <class 'list'>
Cast: <class 'list'>
